**извлечение признаков из текста на естественном языке**

кластеризатор текстов

Евгений Борисов borisov.e@solarl.ru

## библиотеки

In [1]:
import re
# import gzip
import numpy as np
import numpy.random as rng
import pandas as pd
from tqdm import tqdm

np.set_printoptions(precision=2) # вывод на печать чисел до 2 знака
pd.options.display.max_colwidth = 200 

tqdm.pandas()

/usr/lib/python3.8/site-packages/tqdm/std.py:703: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
sklearn.__version__

'0.23.2'

## тексты

In [3]:
# загружаем тексты
data = pd.read_pickle('../data/news.pkl.gz')
print('записей:',len(data))

записей: 3196


In [4]:
data.sample(2)

,text,tag
937,"Макей предложил лидерам России, США, ЕС и Китая встретиться в Минске для откровенного разговора\n\n8 декабря 2016 в 17:36\n\nTUT.BY\n\nЛидерам России, США, ЕС, Китая пора встретиться для откровенн...",economics
1364,"В таблице Менделеева официально появился Московий\n\n30 ноября 2016 в 14:08\n\n42.TUT.BY\n\n28 ноября в таблице Менделеева официально закреплены четыре новых названия — нихоний, московий, теннесси...",tech


## токенизация и очистка

In [5]:
# NOUN	имя существительное	хомяк
# ADJF	имя прилагательное (полное)	хороший
# ADJS	имя прилагательное (краткое)	хорош
# COMP	компаратив	лучше, получше, выше
# VERB	глагол (личная форма)	говорю, говорит, говорил
# INFN	глагол (инфинитив)	говорить, сказать
# PRTF	причастие (полное)	прочитавший, прочитанная
# PRTS	причастие (краткое)	прочитана
# GRND	деепричастие	прочитав, рассказывая
# NUMR	числительное	три, пятьдесят
# ADVB	наречие	круто
# NPRO	местоимение-существительное	он
# PRED	предикатив	некогда
# PREP	предлог	в
# CONJ	союз	и
# PRCL	частица	бы, же, лишь
# INTJ	междометие	ой

In [6]:
# https://pymorphy2.readthedocs.io
from pymorphy2 import MorphAnalyzer

# лематизация и очистка с помощью пакета морфологического анализа

morph = MorphAnalyzer()

def word_normalizer(w, pos_types=('NOUN','VERB','ADJF')):
    if not morph.word_is_known(w): return ''
    p = morph.parse(w)[0] 
    return p.normal_form if (p.tag.POS in pos_types) else ''

def string_normalizer(s):
    return re.sub(r'[\W]+',' ',s.lower().replace('ё','е'))

def text_normalizer(s):
    return [ word_normalizer(w) for w in string_normalizer(s).split() ]

In [7]:
data['ctext'] = data['text'].progress_apply(text_normalizer).progress_apply(' '.join)

100%|██████████| 3196/3196 [00:00<00:00, 133448.10it/s]


In [8]:
data.sample(10)

,text,tag,ctext
1760,"Boeing сократит выпуск лайнеров Boeing 777 из-за низкого спроса\n\nИз-за снижения спроса Boeing сократит производство лайнеров Boeing 777 с августа 2017 г. с нынешних 8,3 единицы в месяц до 5 един...",economics,сократить выпуск лайнер низкий спрос снижение спрос сократить производство лайнер август год нынешний единица месяц единица месяц сообщать интерфакс ссылка пресс релиз концер...
2014,"(обновлено 10:04 13.12.2016 )\n\nМОСКВА, 13 дек - Р-Спорт. Южнокорейские скелетонисты намерены бойкотировать чемпионат мира-2017 по бобслею и скелетону в Сочи из-за допингового скандала, сообщает ...",sport,москва дек спорт южнокорейский чемпионат мир бобслей скелетон сочи допинговый скандал сообщать агентство ссылка слово чиновник корейский федерация бобслей скелетон чемпионат...
3021,"В Алтайском крае будет реализован инвестиционный проект по строительству малых ГЭС. Подписано соответствующее соглашение между главой региона и компаниями <object>МРСК Сибири</object>, Алтайэнерго...",economics,алтайский край быть инвестиционный проект строительство малое гэс соответствующий соглашение главый регион компания сибирь инжиниринговый компания энергия настоящий момент предварите...
2762,"Если сведения источников верны, Apple вновь отступит от своих принципов и выпустит iPhone с двумя SIM-картами.\n\n\n\nИнформации об iPhone 2017 года в последние дни поступает много. И информация э...",tech,сведение источник отступить свой принцип выпустить карта информация год последний день поступать информация этот противоречивый напомнить день японский блог выдать неожиданны...
2464,"В штате Теннеси произошла трагедия, пятилетний мальчик скончался на руках у Санта-Клауса. Об этом сообщили западные СМИ.\n\nАлександр Ельников\n\nМужчина сказал, что он пришел после работы домой и...",culture,штат произойти трагедия мальчик скончаться рука клаус сообщить западный сми александр ельник мужчина сказать прийти работа пара минута раздаться телефонный звонок позвонить больниц...
2709,"На разработку нового сверхпрочного ноутбука Минпромторг перевел 150 млн рублей компании «Т-Платформы». Такой аппарат будет полезен при геологических экспедициях, в нефтегазовой отрасли.\n\nДенис Д...",tech,разработка новое сверхпрочный ноутбук перевести миллион рубль компания том платформа такой аппарат быть геологический экспедиция нефтегазовый отрасль денис давыдов предполагаться устройство...
1655,Россия выплатит компенсацию одному из фигурантов дела Немцова\n\nМосква. 13 декабря. INTERFAX.RU - Россия выплатит одному из обвиняемых по делу об убийстве Бориса Немцова Темирлану Эскерханову шес...,politics,россия выплатить компенсация один фигурант дело немцов москва декабрь россия выплатить один дело убийство борис немцов тысяча евро тот отозвать жалоба европейский суд право человек с...
409,В Бразилии собирают деньги на реставрацию статуи Христа-Искупителя\n\n10 декабря 2016 в 9:09\n\nГазета.Ru\n\nВ Рио-де-Жанейро началась кампания по сбору средств на сохранение статуи Христа-Искупит...,politics,бразилия собирать деньга реставрация статуя христос искупитель декабрь газета начаться кампания сбор средство сохранение статуя христос искупитель сообщать фото издание отмечать...
1559,"Российский Центр примирения сторон продолжает оказывать помощь населению Восточного Алеппо. За последние сутки из этой части города эвакуировали 8 тысяч жителей, из них почти четыре тысячи — дети....",politics,российский центр примирение сторона продолжать помощь население восточный последний сутки этот часть город эвакуировать тысяча житель тысяча ребёнок человек размещать центр помощь имя п...
1951,Сенатор Алексей Пушков прокомментировал ситуацию с пересчетом голосов на выборах президента США.\n\nПредседатель Комиссии Совета Федерации по информационной политике РФ Алексей Пушков высказался о...,politics,сенатор алексей пушок прокомментировать ситуация пересчёт голос выборы президент сша председатель комиссия совет федерация информационный политика рф алексей пушок высказаться итог пересчёт

##  CountVectorizer + TF

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

tf = TfidfVectorizer(use_idf=True,norm='l2').fit( data['ctext'] )

In [10]:
len(tf.vocabulary_)

22128

## формируем датасет

In [11]:
X = tf.transform( data['ctext'] )
X.shape

(3196, 22128)

## обучаем

In [94]:
from sklearn.cluster import DBSCAN

clust = DBSCAN(eps=.95,min_samples=10).fit(X)

CPU times: user 266 ms, sys: 40.1 ms, total: 306 ms
Wall time: 305 ms


In [95]:
data['cluster'] = clust.labels_

In [96]:
data['cluster'].drop_duplicates().count()

36

## тестируем

In [97]:
from matplotlib import pyplot as plt

In [98]:
cluster_descr = pd.concat([
        data[['cluster','tag']].groupby(['cluster'])['tag'].count(),
        data[['cluster','tag']].groupby(['cluster'])['tag'].apply(lambda s: set(s)).apply(' '.join)
    ],axis=1).reset_index()

cluster_descr.columns = ['cluster','count','tags']

cluster_descr

,cluster,count,tags
0,-1,2658,politics incident auto science culture economics health social realty tech sport woman reclama
1,0,17,politics
2,1,21,politics
3,2,15,politics
4,3,34,politics
5,4,28,politics
6,5,35,politics
7,6,13,incident
8,7,25,politics
9,8,12,sport


In [99]:
n = 0
print(cluster_descr.query(f'cluster=={n}'))
data.query(f'cluster=={n}')[['text','tag']].sample(5)

   cluster  count      tags
1        0     17  politics


,text,tag
1680,© Сергей Бобылев/ТАСС\n\nВ декабре 1991 года распался Советский Союз. 25 декабря первый президент СССР Михаил Горбачев сложил с себя полномочия главы государства.\n\nВ преддверии 25-летия этой дат...,politics
1676,"МОСКВА, 13 декабря. /ТАСС/. Экс-президент СССР Михаил Горбачев допускает возможность появления в границах бывшего Советского Союза нового Союзного государства.\n\n""Советского Союза - нет, а Союза ...",politics
1403,"Горбачев заявил о возможном создании в границах бывшего СССР нового Союза\n\nТелеканал «Россия – Вологда»\n\nЭкс-президент Советского Союза Михаил Горбачев заявил, что в границах бывшего СССР може...",politics
1674,"Экс-президент СССР Михаил Горбачев заявил, что в границах Союза может появиться новое объединение, в которое добровольно войдут бывшие советские республики.\n\nОн отметил, что в качестве возврата ...",politics
1673,"Горбачев увидел возможность восстановления Союза в прежних границах\n\nОбъединение, по его словам, может стать полностью добровольным\n\nПервый президент СССР Михаил Горбачев заявил о возможности ...",politics


In [100]:
n = 1
print(cluster_descr.query(f'cluster=={n}'))
data.query(f'cluster=={n}')[['text','tag']].sample(5)

   cluster  count      tags
2        1     21  politics


,text,tag
1597,"Ранее глава Хорватии заявила, что Россия активно влияет на события на Балканах, следствием чего может стать военный конфликт в регионе\n\nМосква, , 09:56 — REGNUM Глава МИД РФ Сергей Лавров заявил...",politics
1596,"Глава МИДа Сергей Лавров прокомментировал заявление президента Хорватии Колинды Грабар-Китарович о том, что Россия вовлечена в военный конфликт на Балканах. Дипломат заверил, что Москва настроена ...",politics
1373,"Глава МИД России Сергей Лавров заявил, что высказывания о возможной российской угрозе на Балканском полуострове странно слышать от хорватских властей, передает РИА «Новости».\n\nРанее президент Хо...",politics
1842,"Заявления о возможной российской угрозе странно слышать от властей Хорватии, однако Москва готова обсудить данные аргументы Загреба, заявил российский министр иностранных дел Сергей Лавров в интер...",politics
1846,"Министр иностранных дел России Сергей Лавров заявил, что высказывания об угрозе конфликта с РФ на Балканах странно слышать от Хорватии, но Москва готова обсудить факты Загреба при их наличии.\n\nР...",politics


In [101]:
n = 2
print(cluster_descr.query(f'cluster=={n}'))
data.query(f'cluster=={n}')[['text','tag']].sample(5)

   cluster  count      tags
3        2     15  politics


,text,tag
1387,"НЬЮ-ЙОРК, 13 декабря. /Корр. ТАСС Кирилл Волков/. Избранный президент США Дональд Трамп провел телефонный разговор с экс-кандидатом на выборах главы государства Миттом Ромни, сообщив ему, что он н...",politics
1383,"Экс-кандидат в президенты США от республиканцев Митт Ромни выбыл из списка претендентов на пост госсекретаря в администрации Дональда Трампа.\r\n\r\n\r\nПо информации телеканала CNN, миллиардер со...",politics
1864,"Избранный президент США Дональд Трамп в ходе телефонного разговора с главным кандидатом на пост госсекретаря Миттом Ромни сообщил, что Ромни не будет занимать эту должность.\n\nПосле разговора экс...",politics
1386,"Дональд Трамп позвонил Ромни и уведомил, что тот не рассматривается в качестве претендента на эту должность. Об этом сообщил телеканал CNN, а затем информацию подтвердил сам Митт Ромни в Facebook....",politics
1381,Место главного дипломата Америки точно не светит экс-кандидату на пост президента США в 2012 году Митту Ромни. С мечтой возглавить дипломатический корпус известный своими антироссийскими высказыва...,politics


In [102]:
n = 3
print(cluster_descr.query(f'cluster=={n}'))
data.query(f'cluster=={n}')[['text','tag']].sample(5)

   cluster  count      tags
4        3     34  politics


,text,tag
1870,"Во вторник, 13 декабря, МИД Японии официально объявил программу визита в страну президента РФ Владимира Путина.\n\nКак стало известно, глава нашего государства прибудет в аэропорт префектуры Ямагу...",politics
1414,"В МИД Японии обнародовали программу российского президента во время рабочей поездки.\n\nПутин и Абэ примут участие в ""бизнес-диалоге""\n\nЯпонское внешнеполитическое ведомство озвучило программу пр...",politics
1572,"Российский лидер Владимир Путин назвал отсутствие мирного договора с Японией анахронизмом, который должен быть устранен.\n\nВ преддверии своего визита президент дал интервью японской телекомпании ...",politics
1873,"Министерство иностранных дел Японии объявило программу пребывания президента России Владимира Путина в стране - визит российского лидера запланирован на 15 и 16 декабря. Предполагается, что сразу ...",politics
1852,"Россия хочет нормализации отношений с Японией, заявил президент России Владимир Путин в интервью японской телекомпании Nippon TV (NTV).\n\n""Начиная с 1956 года, когда были восстановлены дипломатич...",politics
